# 神经网络模型：

In [81]:
import torch

class Para:
    # tensor_board_log_dir = 'runs/exp0'
    feature_column_start_name = 'ep_ratio_ttm'
    feature_column_end_name = 'BR'

    # 模型设置
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification = 2 # 2, 3

    # 权重
    cross_weight = list()
    if classification == 3:
        cross_weight = [1.0, 1.0 ,1.0]
    elif classification == 2:
        cross_weight = [1.0, 1.0]
    elif classification == 5:
        cross_weight = [1.0, 1.0, 1.0, 1.0, 1.0]

    batch_size = 16
    lr = 1e-3
    drop = 0.5
    epochs = 75

    # 数据集设置
    month_in_sample = range(0, 1)
    # month_test = range(36, 48)

    percent_cv = 0.1 # 10% cross validation

    data_path = 'data/mt_space_1d_rate_20d_17-21_pre'


    seed = 2022
    torch.manual_seed(seed)

    info_str0 = 'mt_'+'c'+str(classification)+'_s'+str(percent_cv)
    info_str1 = '_b'+str(batch_size)+'_lr'+str(lr)+'_d'+str(drop)+'_e'+str(epochs)
    info_str = info_str0 + info_str1

    save_model_path = 'models/'+'model_'+info_str+'.pth'

para = Para()
print(para.info_str)
print(para.save_model_path)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    print(torch.cuda.current_device())

mt_c2_s0.05_b16_lr0.001_d0.5_e75
models/model_mt_c2_s0.05_b16_lr0.001_d0.5_e75.pth
True
1
GeForce MX150
0


### 构建训练集

In [82]:
import numpy as np
import pandas as pd

data_in_sample = None
for i_month in para.month_in_sample:
    file_name = para.data_path + '/' + str(i_month) + '.csv'
    data_curr_month = pd.read_csv(file_name)

    data_curr_month = data_curr_month.dropna(axis=0)

    data_curr_month.insert(loc=0, column='return_bin', value=np.nan)

    data_curr_month.loc[data_curr_month['yield_rate']>0, 'return_bin'] = 0
    data_curr_month.loc[data_curr_month['yield_rate']<=0, 'return_bin'] = 1

    if i_month == para.month_in_sample[0]:
        data_in_sample = data_curr_month
    else:
        data_in_sample = pd.concat([data_in_sample, data_curr_month])
        # data_in_sample = data_in_sample.append(data_curr_month)

data_in_sample

,return_bin,order_book_id,board_type,sector_code,month,date,yield_rate,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,...,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.0,600519.XSHG,MainBoard,ConsumerStaples,0,2017-01-03,-0.092587,1.582679,-1.683954,1.603209,...,0.401807,0.988619,0.061485,-0.671471,-1.318601,-1.576493,-1.136058,-1.217233,0.560313,0.139982
1,1.0,600519.XSHG,MainBoard,ConsumerStaples,1,2017-01-04,-0.645762,1.302261,-1.583147,1.336612,...,1.080383,0.988619,1.075444,-0.532821,-1.266669,-1.548342,-1.060905,-1.199013,1.141963,0.551387
2,1.0,600519.XSHG,MainBoard,ConsumerStaples,2,2017-01-05,-0.369527,1.382886,-1.613186,1.413263,...,0.609067,0.988619,0.685052,-0.505367,-1.217169,-1.570874,-1.030676,-1.189542,0.668555,0.023353
3,1.0,600519.XSHG,MainBoard,ConsumerStaples,3,2017-01-06,-0.581350,1.319990,-1.589829,1.353466,...,0.988294,1.719976,0.894761,-0.401765,-1.102462,-1.549486,-0.995422,-1.178730,1.176819,0.547457
4,1.0,600519.XSHG,MainBoard,ConsumerStaples,4,2017-01-09,-0.357646,1.355014,-1.602902,1.386764,...,1.230286,0.988619,0.717882,-0.355223,-1.052310,-1.575254,-0.972552,-1.170599,1.197702,0.310576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,0.0,600519.XSHG,MainBoard,ConsumerStaples,1192,2021-11-29,0.439775,-1.199925,0.939052,-1.223484,...,1.741930,0.988619,1.417920,-0.941589,-0.287143,0.235562,1.221078,0.919459,0.760456,0.329757
1190,0.0,600519.XSHG,MainBoard,ConsumerStaples,1193,2021-11-30,0.818726,-1.118328,0.814286,-1.150999,...,0.771590,0.988619,0.726018,-0.953039,-0.285410,0.253470,1.322049,0.900321,0.468537,0.267050
1191,0.0,600519.XSHG,MainBoard,ConsumerStaples,1194,2021-12-01,0.234917,-1.121759,0.819394,-1.154046,...,0.957610,1.719976,0.651600,-0.961177,-0.338331,0.246088,1.344969,0.868925,0.363120,0.467004
1192,0.0,600519.XSHG,MainBoard,ConsumerStaples,1195,2021-12-02,0.427783,-1.123316,0.821717,-1.155430,...,1.004908,1.719976,0.586634,-0.969949,-0.386910,0.238948,1.402479,0.808674,0.583062,0.834642


In [83]:
X_in_sample = data_in_sample.loc[:, para.feature_column_start_name: para.feature_column_end_name]
y_in_sample = data_in_sample.loc[:, 'return_bin']

from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False) # True, random_state=para.seed)
# X_train, X_cv, y_train, y_cv = train_test_split(X_in_sample, y_in_sample, test_size=para.percent_cv, shuffle=False)

X_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR
0,1.582679,-1.683954,1.603209,-0.237438,-0.255987,-0.006208,0.401807,0.988619,0.061485,-0.671471,-1.318601,-1.576493,-1.136058,-1.217233,0.560313,0.139982
1,1.302261,-1.583147,1.336612,-0.175133,-0.241780,0.156462,1.080383,0.988619,1.075444,-0.532821,-1.266669,-1.548342,-1.060905,-1.199013,1.141963,0.551387
2,1.382886,-1.613186,1.413263,-0.143321,-0.223657,0.205494,0.609067,0.988619,0.685052,-0.505367,-1.217169,-1.570874,-1.030676,-1.189542,0.668555,0.023353
3,1.319990,-1.589829,1.353466,-0.108782,-0.201822,0.251971,0.988294,1.719976,0.894761,-0.401765,-1.102462,-1.549486,-0.995422,-1.178730,1.176819,0.547457
4,1.355014,-1.602902,1.386764,-0.090909,-0.180558,0.244829,1.230286,0.988619,0.717882,-0.355223,-1.052310,-1.575254,-0.972552,-1.170599,1.197702,0.310576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,-0.589585,0.188710,-0.664421,-2.999583,-3.350904,0.498992,-1.067853,-1.205453,-1.299105,1.649936,0.536805,0.242242,1.009761,2.918412,-1.041683,-1.499852
1130,-0.591854,0.191175,-0.666447,-2.940842,-3.305178,0.551141,-1.157597,-0.474096,-1.236947,1.652259,0.562759,0.204025,1.089944,2.926244,-0.942727,-1.432810
1131,-0.569636,0.167175,-0.646604,-2.894573,-3.258768,0.559691,-1.206234,-0.474096,-1.261473,1.725005,0.604266,0.187668,1.069271,2.911454,-0.960339,-1.406643
1132,-0.506099,0.100177,-0.589857,-2.911356,-3.225205,0.398826,-0.998123,-0.474096,-1.499523,1.793247,0.641807,0.134481,1.086997,2.886990,-0.717884,-1.238148


In [84]:
y_train

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
1129    0.0
1130    0.0
1131    0.0
1132    0.0
1133    0.0
Name: return_bin, Length: 1134, dtype: float64

In [85]:
data_train = pd.concat([X_train, y_train], axis=1)
data_cv = pd.concat([X_cv, y_cv], axis=1)
data_train

,ep_ratio_ttm,pb_ratio_ttm,sp_ratio_ttm,MACD_DIFF,MACD_DEA,MACD_HIST,RSI10,SY,BIAS20,VOL30,VOL60,VOL120,VOLT20,VOLT60,AR,BR,return_bin
0,1.582679,-1.683954,1.603209,-0.237438,-0.255987,-0.006208,0.401807,0.988619,0.061485,-0.671471,-1.318601,-1.576493,-1.136058,-1.217233,0.560313,0.139982,1.0
1,1.302261,-1.583147,1.336612,-0.175133,-0.241780,0.156462,1.080383,0.988619,1.075444,-0.532821,-1.266669,-1.548342,-1.060905,-1.199013,1.141963,0.551387,1.0
2,1.382886,-1.613186,1.413263,-0.143321,-0.223657,0.205494,0.609067,0.988619,0.685052,-0.505367,-1.217169,-1.570874,-1.030676,-1.189542,0.668555,0.023353,1.0
3,1.319990,-1.589829,1.353466,-0.108782,-0.201822,0.251971,0.988294,1.719976,0.894761,-0.401765,-1.102462,-1.549486,-0.995422,-1.178730,1.176819,0.547457,1.0
4,1.355014,-1.602902,1.386764,-0.090909,-0.180558,0.244829,1.230286,0.988619,0.717882,-0.355223,-1.052310,-1.575254,-0.972552,-1.170599,1.197702,0.310576,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1129,-0.589585,0.188710,-0.664421,-2.999583,-3.350904,0.498992,-1.067853,-1.205453,-1.299105,1.649936,0.536805,0.242242,1.009761,2.918412,-1.041683,-1.499852,0.0
1130,-0.591854,0.191175,-0.666447,-2.940842,-3.305178,0.551141,-1.157597,-0.474096,-1.236947,1.652259,0.562759,0.204025,1.089944,2.926244,-0.942727,-1.432810,0.0
1131,-0.569636,0.167175,-0.646604,-2.894573,-3.258768,0.559691,-1.206234,-0.474096,-1.261473,1.725005,0.604266,0.187668,1.069271,2.911454,-0.960339,-1.406643,0.0
1132,-0.506099,0.100177,-0.589857,-2.911356,-3.225205,0.398826,-0.998123,-0.474096,-1.499523,1.793247,0.641807,0.134481,1.086997,2.886990,-0.717884,-1.238148,0.0


In [86]:
from torch.utils.data import TensorDataset
import torch

X_train_ndarray = X_train.values
y_train_ndarray = y_train.values

train_dataset = TensorDataset(torch.from_numpy(X_train_ndarray).type(torch.FloatTensor), torch.from_numpy(y_train_ndarray).type(torch.LongTensor))

# for X_train_temp, y_train_temp in train_dataset:
#     print(X_train_temp, y_train_temp)
#     print(X_train_temp.dtype, y_train_temp.dtype)

In [87]:
X_cv_ndarray = X_cv.values
y_cv_ndarray = y_cv.values

cv_dataset = TensorDataset(torch.from_numpy(X_cv_ndarray).type(torch.FloatTensor), torch.from_numpy(y_cv.values).type(torch.LongTensor))

# for X_cv_temp, y_cv_temp in cv_dataset:
#     print(X_cv_temp, y_cv_temp)
#     print(X_cv_temp.dtype, y_cv_temp.dtype)

In [88]:
from torch.utils.data import DataLoader


train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

cv_dataloader = DataLoader(
    dataset=cv_dataset,
    batch_size=para.batch_size,
    shuffle=True
)

### 构建测试集

In [89]:
# data_test = None
# for i_month in para.month_test:
#
#     file_name = para.data_path + '/' + str(i_month) + '.csv'
#     data_curr_month = pd.read_csv(file_name)
#
#     data_curr_month = data_curr_month.dropna(axis=0)
#
#     data_curr_month = label_data(data=data_curr_month, percent_select=para.percent_select)
#
#     if i_month == para.month_test[0]:
#         data_test = data_curr_month
#     else:
#         data_test = pd.concat([data_test, data_curr_month])
#         # data_test = data_test.append(data_curr_month)
#
# X_test = data_test.loc[:, para.feature_column_start_name: para.feature_column_end_name]
# y_test = data_test.loc[:, 'return_bin']

In [90]:
# from torch.utils.data import TensorDataset
# import torch
#
#
# X_test_ndarray = X_test.values
# y_test_ndarray = y_test.values
#
# test_dataset = TensorDataset(torch.from_numpy(X_test_ndarray).type(torch.FloatTensor), torch.from_numpy(y_test_ndarray).type(torch.LongTensor))

In [91]:
# from torch.utils.data import DataLoader
#
#
# test_dataloader = DataLoader(
#     dataset=test_dataset,
#     batch_size=para.batch_size,
#     shuffle=True,
# )

### 构建神经网络

In [92]:
from my_utils.model_class import MLP

model = MLP(in_nums=len(X_train.columns), out_nums=para.classification, drop_p=para.drop)
# to device
model = model.to(device=para.device)
print(model)

MLP(
  (linear_stack): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): Linear(in_features=16, out_features=8, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=2, bias=True)
  )
)


### 训练与测试

In [93]:
import torchmetrics


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # initialize metric
    train_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    for batch, (X, y) in enumerate(dataloader):

        # to device
        X = X.to(device=para.device)
        y = y.to(device=para.device)

        # compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        train_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        # metric on current batch
        train_precision(pred.argmax(1), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    train_loss /= num_batches
    correct /= size
    print(f"Train Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {train_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = train_precision.compute()
    print("Precision of every train dataset class: ", total_precision)
    print()

    return correct, train_loss, total_precision


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # initialize metric
    test_precision = torchmetrics.Precision(average='none', num_classes=para.classification).to(device=para.device)

    with torch.no_grad():
        for X, y in dataloader:

            # to device
            X = X.to(device=para.device)
            y = y.to(device=para.device)

            # compute prediction and loss
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            # metric on current batch
            test_precision(pred.argmax(1), y)


    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n    Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    # metric on all batches using custom accumulation
    total_precision = test_precision.compute()
    print("Precision of every test dataset class: ", total_precision)
    print()

    return correct, test_loss, total_precision

In [94]:
def select_df_to_dataloader(df: pd.DataFrame, select: int) -> DataLoader:

    df = df[df['return_bin'] == select]

    df_dataset = TensorDataset(
        torch.from_numpy(df.loc[:, para.feature_column_start_name: para.feature_column_end_name].values).type(torch.FloatTensor),
        torch.from_numpy(df.loc[:, 'return_bin'].values).type(torch.LongTensor))

    df_dataloader = DataLoader(
        dataset=df_dataset,
        batch_size=para.batch_size,
        shuffle=True,
    )

    return df_dataloader

In [95]:
# temp2_dataloader = select_df_to_dataloader(df=data_cv, select=2)
temp1_dataloader = select_df_to_dataloader(df=data_cv, select=1)
temp0_dataloader = select_df_to_dataloader(df=data_cv, select=0)

In [96]:
from torch import nn
import time
from torch.utils.tensorboard import SummaryWriter

# 计时
time_start = time.time()

# writer = SummaryWriter(para.tensor_board_log_dir)
writer = SummaryWriter()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
# to device
loss_fn = loss_fn.to(device=para.device)

# 优化器
optimizer = torch.optim.Adam(model.parameters(), lr=para.lr)

epochs = para.epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")

    model.train()
    accuracy_train, loss_train, precision_train = train_loop(train_dataloader, model, loss_fn, optimizer)
    model.eval()
    accuracy_cv, loss_cv, precision_cv = test_loop(cv_dataloader, model, loss_fn)

    # accuracy2 = test_loop(temp2_dataloader, model, loss_fn)
    # print('#')
    # accuracy1 = test_loop(temp1_dataloader, model, loss_fn)
    # accuracy0 = test_loop(temp0_dataloader, model, loss_fn)

    # 写入 tensorboard
    if para.classification == 2:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1]},
                           global_step=t)

    elif para.classification == 3:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2]},
                           global_step=t)

    elif para.classification == 5:

        writer.add_scalars(main_tag=para.info_str+'_evaluation/cv',
                           tag_scalar_dict={
                               'accuracy': accuracy_cv,
                               'precision0': precision_cv[0],
                               'precision1': precision_cv[1],
                               'precision2': precision_cv[2],
                               'precision3': precision_cv[3],
                               'precision4': precision_cv[4]},
                           global_step=t)

        writer.add_scalars(main_tag=para.info_str+'_evaluation/train',
                           tag_scalar_dict={
                               'accuracy': accuracy_train,
                               'precision0': precision_train[0],
                               'precision1': precision_train[1],
                               'precision2': precision_train[2],
                               'precision3': precision_train[3],
                               'precision4': precision_train[4]},
                           global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/cv',
                       tag_scalar_dict={
                           'loss': loss_cv},
                       global_step=t)

    writer.add_scalars(main_tag=para.info_str+'_loss/train',
                       tag_scalar_dict={
                           'loss': loss_train},
                       global_step=t)
    writer.flush()

    time_end = time.time()
    print('Time cost = %fs' % (time_end - time_start))
    print()

writer.close()

print("Done!")

Epoch 1
-------------------------------
Train Error: 
    Accuracy: 53.6%, Avg loss: 0.689240 

Precision of every train dataset class:  tensor([0.5302, 0.5400], device='cuda:0')

Test Error: 
    Accuracy: 28.3%, Avg loss: 0.715230 

Precision of every test dataset class:  tensor([0.0000, 0.2833], device='cuda:0')

Time cost = 0.965886s

Epoch 2
-------------------------------
Train Error: 
    Accuracy: 53.9%, Avg loss: 0.686479 

Precision of every train dataset class:  tensor([0.5375, 0.5395], device='cuda:0')

Test Error: 
    Accuracy: 28.3%, Avg loss: 0.724632 

Precision of every test dataset class:  tensor([0.0000, 0.2833], device='cuda:0')

Time cost = 1.808232s

Epoch 3
-------------------------------
Train Error: 
    Accuracy: 53.4%, Avg loss: 0.682902 

Precision of every train dataset class:  tensor([0.5352, 0.5340], device='cuda:0')

Test Error: 
    Accuracy: 28.3%, Avg loss: 0.738721 

Precision of every test dataset class:  tensor([0.0000, 0.2833], device='cuda:0')



## 保存模型

In [97]:
torch.save(model.state_dict(), para.save_model_path)

print('Finish save model!')

Finish save model!


## captum

In [98]:
# # captum
# from captum.attr import IntegratedGradients
#
# ig = IntegratedGradients(model)

In [99]:
# temp = cv_dataloader.dataset.tensors[0]
# temp.requires_grad_()
# attr, delta = ig.attribute(temp,target=1, return_convergence_delta=True)
# attr = attr.detach().numpy()

In [100]:
# # Helper method to print importances and visualize distribution
# def visualize_importances(feature_names, importances, title="Average Feature Importances", plot=True, axis_title="Features"):
#     print(title)
#     for i in range(len(feature_names)):
#         print(feature_names[i], ": ", '%.3f'%(importances[i]))
#     y_pos = (np.arange(len(feature_names)))
#     if plot:
#         plt.figure(figsize=(20,6))
#         plt.barh(y_pos, importances, align='center')
#         plt.yticks(y_pos, feature_names)
#         plt.ylabel(axis_title)
#         plt.grid(axis='y')
#         plt.title(title)
# visualize_importances(feature_names=X_cv.columns.values.tolist(), importances=np.mean(attr, axis=0))

In [101]:
# X_cv.columns.values.tolist()

In [102]:
# loss = nn.CrossEntropyLoss()
# input = torch.Tensor(
#     [[-0.0441,  0.0773],
#     [-0.0781, -0.1772],
#     [-0.1319, -0.0432],
#     [-0.0714, -0.1261],
#     [-0.0806, -0.1370],
#     [-0.1730, -0.1472],
#     [-0.0350, -0.0507],
#     [-0.1149, -0.2248]])
# # input = input.reshape(-1,4)
# target = torch.Tensor([0, 1, 1, 0, 0, 0, 0, 0]).type(torch.LongTensor)
# print(input.dtype)
# print(target.dtype)
# output = loss(input, target)
# print(input, target, output)

In [103]:
# # Example of target with class indices
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.Tensor([1,4,1]).type(torch.LongTensor)
# output = loss(input, target)
# print(input,target,output)

In [104]:
# loss = nn.BCEWithLogitsLoss()
# input = torch.Tensor([0.5, 0.4, 0.3])
# target = torch.Tensor([0])
# output = loss(input, target)
# print(input, target, output)